In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
with open("../../circuit_config_2.json", "r") as f:
    configs = json.load(f)

In [4]:
import random
save_names = []

print("#!/bin/bash\n")
print("set -e\n")

print("clear; parallel -j8 --ungroup python NeSymReS/DataGeneration/ProbeDatasetGeneration.py --operation {} --idx 1 ::: add cos exp log mul pow sin tan")

for config in configs[:]:
    print("# -----------------------------------------")
    filtered_config = {key: value for key, value in config.items() if key != "circuit"}
    print(f"# Config: {filtered_config}")
    magic_number = 500*2 if config["operation"] != 'log' else 309*2
    # Sampling indices
    circuit = set(config["circuit"])
    full_range = set(range(0, 117))
    remove = set(range(0, 117, 9))
    circuit -= remove
    complement = full_range - circuit - remove

    random.seed(0)
    circuit_sample = random.sample(list(circuit), min(10, len(circuit)))
    complement_sample = random.sample(list(complement), min(10, len(complement)))
    selected_indices = circuit_sample + complement_sample
    selected_indices_str = ",".join(str(idx) for idx in selected_indices)
    selected_indices_space = " ".join(str(idx) for idx in selected_indices)

    # Create safe save name
    save_name = "results_" + "_".join(f"{k}-{v}" for k, v in filtered_config.items())
    save_names.append("/home/arco/Downloads/Master/MscThesis/ExplainableDSR/data/Arco/Probing/" + save_name)
    
    # Print dataset generation command
    print(f"parallel -j10 --ungroup python NeSymReS/DataGeneration/ProbeDatasetGeneration.py "
          f"--operation {config['operation']} --idx ::: {selected_indices_space}")

    # Print probing experiment command
    print(f"python NeSymReS/Probing/run_experiment.py "
          f"--indices {selected_indices_str} "
          f"--datapath /home/arco/Downloads/Master/MscThesis/ExplainableDSR/data/Arco/Datasets/Probing/cached_values_{magic_number}_{config['operation']}_correct_ "
          f"--save {save_name}\n")

# print(save_names)

#!/bin/bash

set -e

clear; parallel -j8 --ungroup python NeSymReS/DataGeneration/ProbeDatasetGeneration.py --operation {} --idx 1 ::: add cos exp log mul pow sin tan
# -----------------------------------------
# Config: {'operation': 'add', 'patch_type': 'resample', 'CTR_token': 'cos', 'CTE': 'cos-tan', 'CTR': False, 'Evaluation_type': 'functional'}
parallel -j10 --ungroup python NeSymReS/DataGeneration/ProbeDatasetGeneration.py --operation add --idx ::: 78 80 15 56 94 93 79 59 92 76 66 23 53 12 33 103 87 7 68 31
python NeSymReS/Probing/run_experiment.py --indices 78,80,15,56,94,93,79,59,92,76,66,23,53,12,33,103,87,7,68,31 --datapath /home/arco/Downloads/Master/MscThesis/ExplainableDSR/data/Arco/Datasets/Probing/cached_values_1000_add_correct_ --save results_operation-add_patch_type-resample_CTR_token-cos_CTE-cos-tan_CTR-False_Evaluation_type-functional

# -----------------------------------------
# Config: {'operation': 'add', 'patch_type': 'resample', 'CTR_token': 'cos', 'CTE': 'cos

In [ ]:
from scipy.stats import ttest_ind, ttest_rel
import numpy as np

for config, name in zip(configs, save_names):
    try:
        accs = np.load(f"{name}.npy", allow_pickle=True)
        accs = [a[1] for a in accs]
        cir = np.array(accs[0:10])
        com = np.array(accs[10:])

        cir_means = cir.mean(axis=1)
        com_means = com.mean(axis=1)
    except:
        print(f"could not load {name}\n")
        continue
    

    # Paired t-test: tests if the means for corresponding heads are significantly different
    t_stat, p_value = ttest_rel(cir_means, com_means)
    cir = np.array([x[0] if isinstance(x, list) else x for x in cir])
    com = np.array([x[0] if isinstance(x, list) else x for x in com])
    t_stat, p_value = ttest_ind(np.sort(com), np.sort(cir), equal_var=False)
    p_value = p_value.mean().item()

    print(f"{config['operation']} & {config['patch_type'][0].capitalize()}{config['Evaluation_type'][0].capitalize()} & {cir.mean()/100:.2f} & {cir.std()/100:.2f} && {com.mean()/100:.2f} & {com.std()/100:.2f} & {p_value:.3f}")

add & RF & 0.95 & 0.05 && 0.90 & 0.09 & 0.208
add & RM & 0.94 & 0.05 && 0.90 & 0.08 & 0.236
cos & MF & 0.98 & 0.01 && 0.97 & 0.01 & 0.402
cos & MM & 0.97 & 0.01 && 0.97 & 0.01 & 0.541
cos & RF & 0.97 & 0.01 && 0.97 & 0.01 & 0.764
cos & RM & 0.97 & 0.01 && 0.97 & 0.01 & 0.495
cos & RF & 0.97 & 0.01 && 0.97 & 0.01 & 0.532
exp & MF & 0.96 & 0.02 && 0.95 & 0.02 & 0.186
exp & MM & 0.96 & 0.02 && 0.95 & 0.02 & 0.617
log & MF & 0.96 & 0.02 && 0.88 & 0.08 & 0.015
log & MM & 0.96 & 0.02 && 0.88 & 0.09 & 0.029
log & RF & 0.93 & 0.06 && 0.90 & 0.09 & 0.404
log & RM & 0.96 & 0.02 && 0.90 & 0.08 & 0.051
log & RF & 0.90 & 0.07 && 0.90 & 0.08 & 0.938
mul & RF & 0.92 & 0.03 && 0.90 & 0.04 & 0.206
mul & RM & 0.91 & 0.04 && 0.89 & 0.04 & 0.189
pow & MF & 0.98 & 0.02 && 0.98 & 0.01 & 0.482
pow & MM & 0.98 & 0.02 && 0.98 & 0.01 & 0.628
sin & MF & 0.94 & 0.03 && 0.93 & 0.03 & 0.661
sin & MM & 0.94 & 0.03 && 0.93 & 0.03 & 0.565
sin & RF & 0.93 & 0.03 && 0.93 & 0.03 & 0.878
sin & RM & 0.93 & 0.03 && 0.94 & 0